In [15]:
import math

In [21]:
def approximate_median(range_list, design_factor=1, sampling_percentage=None, jam_values=None):
    """
    Estimate a median and approximate the margin of error.
    Follows the U.S. Census Bureau's `official guidelines`_ for estimation using a design factor.
    Useful for generating medians for measures like household income and age when aggregating census geographies.
    Args:
        range_list (list): A list of dictionaries that divide the full range of data values into continuous categories.
            Each dictionary should have three keys:
                * min (int): The minimum value of the range
                * max (int): The maximum value of the range
                * n (int): The number of people, households or other unit in the range
        design_factor (float, optional): A statistical input used to tailor the standard error to the
            variance of the dataset. This is only needed for data coming from public use microdata sample,
            also known as PUMS. You do not need to provide this input if you are approximating
            data from the American Community Survey. The design factor for each PUMS
            dataset is provided as part of `the bureau's reference material`_.
        sampling_percentage (float, optional): A statistical input used to correct for variance linked to
            the size of the survey's population sample. This value submitted should be the percentage of
            * One-year PUMS: 1
            * One-year ACS: 2.5
            * Three-year ACS: 7.5
            * Five-year ACS: 12.5
         If you do not provide this input, a margin of error will not be returned.
        jam_values (list of two, optional):  If the median falls in the minimum or maximum bracket, 
            the median cannot be calculated. "Jam values" provided in the `American Community Survey's
            technical documentation`_ can be returned as more informative versions of N/A if known.
    Returns:
        A two-item tuple with the median followed by the approximated margin of error.
        (42211.096153846156, 10153.200960954948)
    Examples:
        Estimating the median for a range of household incomes.
        >>> household_income_2013_acs5 = [
            dict(min=2499, max=9999, n=186),
            dict(min=10000, max=14999, n=78),
            dict(min=15000, max=19999, n=98),
            dict(min=20000, max=24999, n=287),
            dict(min=25000, max=29999, n=142),
            dict(min=30000, max=34999, n=90),
            dict(min=35000, max=39999, n=107),
            dict(min=40000, max=44999, n=104),
            dict(min=45000, max=49999, n=178),
            dict(min=50000, max=59999, n=106),
            dict(min=60000, max=74999, n=177),
            dict(min=75000, max=99999, n=262),
            dict(min=100000, max=124999, n=77),
            dict(min=125000, max=149999, n=100),
            dict(min=150000, max=199999, n=58),
            dict(min=200000, max=250001, n=18)
        ]
        >>> approximate_median(household_income_2013_acs5, sampling_percentage=5*2.5)
        (42211.096153846156, 4706.522752733644)
    ... _official guidelines:
        https://www.documentcloud.org/documents/6165603-2013-2017AccuracyPUMS.html#document/p18
    ... _American Community Survey's technical documentation
        https://www.documentcloud.org/documents/6165752-2017-SummaryFile-Tech-Doc.html#document/p20/a508561
    ... _the bureau's reference material:
        https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.html
    """
    # Sort the list
    #range_list.sort(key=lambda x: x['min'])

    # For each range calculate its min and max value along the universe's scale
    cumulative_n = 0
    for range_ in range_list:
        range_['n_min'] = cumulative_n
        cumulative_n += range_['n']
        range_['n_max'] = cumulative_n

    # What is the total number of observations in the universe?
    n = sum([d['n'] for d in range_list])

    # What is the estimated midpoint of the n?
    n_midpoint = n / 2.0

    # Now use those to determine which group contains the midpoint.
    n_midpoint_range = next(d for d in range_list if n_midpoint >= d['n_min'] and n_midpoint <= d['n_max'])

    # How many households in the midrange are needed to reach the midpoint?
    n_midrange_gap = n_midpoint - n_midpoint_range['n_min']

    # What is the proportion of the group that would be needed to get the midpoint?
    n_midrange_gap_percent = n_midrange_gap / n_midpoint_range['n']
    
    # Apply this proportion to the width of the midrange
    n_midrange_gap_adjusted = (n_midpoint_range['max'] - n_midpoint_range['min']) * n_midrange_gap_percent

    # Estimate the median
    estimated_median = n_midpoint_range['min'] + n_midrange_gap_adjusted

    
    if not jam_values and math.isnan(n_midpoint_range['max']):
        # Let's throw a warning
        warnings.warn("", JamValueWarning)
        return None, None
    
    if not jam_values and math.isnan(n_midpoint_range['min']):
        # Let's throw a warning
        warnings.warn("", JamValueWarning)
        return None, None

    if math.isnan(n_midpoint_range['max']):
        estimated_median = jam_values[1]
        
    elif math.isnan(n_midpoint_range['min']):
        estimated_median = jam_values[0]   

    # If there's no sampling percentage, we can't calculate a margin of error
    if not sampling_percentage:
        # Let's throw a warning, but still return the median
        warnings.warn("", SamplingPercentageWarning)
        return estimated_median, None

    # Get the standard error for this dataset
    standard_error = (design_factor * math.sqrt(((100 - sampling_percentage) / (n * sampling_percentage)) * (50**2))) / 100

    # Use the standard error to calculate the p values
    p_lower = .5 - standard_error
    p_upper = .5 + standard_error

    # Estimate the p_lower and p_upper n values
    p_lower_n = n * p_lower
    p_upper_n = n * p_upper

    # Find the ranges the p values fall within
    try:
        p_lower_range_i, p_lower_range = next(
            (i, d) for i, d in enumerate(range_list)
            if p_lower_n >= d['n_min'] and p_lower_n <= d['n_max']
        )
    except StopIteration:
        raise DataError(f"The n's lower p value {p_lower_n} does not fall within a data range.")

    try:
        p_upper_range_i, p_upper_range = next(
            (i, d) for i, d in enumerate(range_list)
            if p_upper_n >= d['n_min'] and p_upper_n <= d['n_max']
        )
    except StopIteration:
        raise DataError(f"The n's upper p value {p_upper_n} does not fall within a data range.")

    # Use these values to estimate the lower bound of the confidence interval
    p_lower_a1 = p_lower_range['min']
    try:
        p_lower_a2 = range_list[p_lower_range_i + 1]['min']
    except IndexError:
        p_lower_a2 = p_lower_range['max']
    p_lower_c1 = p_lower_range['n_min'] / n
    try:
        p_lower_c2 = range_list[p_lower_range_i + 1]['n_min'] / n
    except IndexError:
        p_lower_c2 = p_lower_range['n_max'] / n
    lower_bound = ((p_lower - p_lower_c1) / (p_lower_c2 - p_lower_c1)) * (p_lower_a2 - p_lower_a1) + p_lower_a1

    # Same for the upper bound
    p_upper_a1 = p_upper_range['min']
    try:
        p_upper_a2 = range_list[p_upper_range_i + 1]['min']
    except IndexError:
        p_upper_a2 = p_upper_range['max']
    p_upper_c1 = p_upper_range['n_min'] / n
    try:
        p_upper_c2 = range_list[p_upper_range_i + 1]['n_min'] / n
    except IndexError:
        p_upper_c2 = p_upper_range['n_max'] / n
    upper_bound = ((p_upper - p_upper_c1) / (p_upper_c2 - p_upper_c1)) * (p_upper_a2 - p_upper_a1) + p_upper_a1

    # Calculate the standard error of the median
    standard_error_median = 0.5 * (upper_bound - lower_bound)

    # Calculate the margin of error at the 90% confidence level
    margin_of_error = 1.645 * standard_error_median
    
    if math.isnan(margin_of_error):
        margin_of_error = None
    
    # Return the result
    return estimated_median, margin_of_error

In [17]:
household_income_2013_acs5 = [
            dict(min=math.nan, max=9999, n=186),
            dict(min=10000, max=14999, n=1),
            dict(min=15000, max=19999, n=1),
            dict(min=20000, max=24999, n=1),
            dict(min=25000, max=29999, n=1),
            dict(min=30000, max=34999, n=1),
            dict(min=35000, max=39999, n=1),
            dict(min=40000, max=44999, n=1),
            dict(min=45000, max=49999, n=1),
            dict(min=50000, max=59999, n=1),
            dict(min=60000, max=74999, n=1),
            dict(min=75000, max=99999, n=2),
            dict(min=100000, max=124999, n=7),
            dict(min=125000, max=149999, n=1),
            dict(min=150000, max=199999, n=5),
            dict(min=200000, max=math.nan, n=18)
        ]

In [14]:
approximate_median(household_income_2013_acs5, sampling_percentage=1)

(nan, nan)

In [22]:
approximate_median(household_income_2013_acs5, sampling_percentage=1, jam_values=[2599, 200001])

(2599, None)

Need to write JamValueWarning

In [ ]:
class JamValueWarning(Warning):
    """
    Warns that you have not provided jam values when they are needed.
    """
    def __str__(self):
        return """The median falls in the upper or lower bracket. Provide a jam value for more information."""
